# 🔄 Silver - tb_pesquisa (SRC)

**Camada**: Silver  
**Catálogo**: v_credit  
**Tabelas de Saída**:
- `tb_pesquisa` → Registros válidos (Inclui pesquisas sem resposta/NULL)
- `tb_pesquisa_invalidos` → Registros rejeitados (auditoria)

**Origem**: `v_credit.bronze.pesquisa_satisfacao`

**Objetivo**: Limpar, padronizar e validar dados de pesquisas de satisfação (NPS/CSAT). 
**Nota**: Valores de nota vazios são mantidos como NULL. Data de ingestão é preservada da origem.

## 📋 Imports e Configuração

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

processing_timestamp = spark.sql("SELECT current_timestamp() as ts").collect()[0]['ts']

In [0]:
# Configurações de Regras de Negócio
# Nota deve ser inteira entre 1 e 5, ou NULA (caso não respondida)
VALID_NOTES = [1, 2, 3, 4, 5]

##1️⃣ Leitura da Bronze

In [0]:
TABLE_SOURCE = "v_credit.bronze.pesquisa_satisfacao"
TABLE_TARGET_VALIDOS = "v_credit.silver.tb_pesquisa"
TABLE_TARGET_INVALIDOS = "v_credit.silver.tb_pesquisa_invalidos"

In [0]:
df_bronze = spark.table(TABLE_SOURCE)

total_bronze = df_bronze.count()
print(f"\n✅ Total de registros lidos da Bronze: {total_bronze}")

## 2️⃣ Limpeza e Padronização

In [0]:
# Aplica casting e renomeação conforme dicionário de dados
df_limpo = (
    df_bronze
    .select(
        F.col("id_pesquisa").cast("bigint").alias("cd_pesquisa"),
        F.col("id_chamado").cast("bigint").alias("cd_chamado"),
        # try_cast converte erro (string vazia) em NULL pacificamente
        F.expr("try_cast(nota_atendimento as smallint)").alias("nu_nota"),
        # CORREÇÃO: Mantém estritamente o timestamp da origem (Bronze)
        F.col("ingestion_timestamp").alias("dt_ingestion"),
        F.coalesce(F.col("origem"), F.lit("sistema_pesquisa")).alias("dc_origem")
    )
    # Remove duplicatas pela Chave Primária
    .dropDuplicates(["cd_pesquisa"])
    .withColumn("data_processamento", F.current_timestamp())
)

total_limpo = df_limpo.count()
print(f"✅ Dados limpos e padronizados: {total_limpo} registros")

## 3️⃣ Regras de Validação e Qualidade

In [0]:
df_validacao = (
    df_limpo
    # Flag 1: PK (cd_pesquisa) não pode ser NULL
    .withColumn(
        "flag_pk_valida", 
        F.col("cd_pesquisa").isNotNull()
    )
    # Flag 2: FK (cd_chamado) não pode ser NULL (garantir vínculo)
    .withColumn(
        "flag_fk_valida", 
        F.col("cd_chamado").isNotNull()
    )
    # Flag 3: Validação da Nota
    # Regra: Deve ser NULL (permitido vazio) OU entre 1 e 5
    .withColumn(
        "flag_nota_valida", 
        (F.col("nu_nota").isNull()) | 
        ((F.col("nu_nota") >= 1) & (F.col("nu_nota") <= 5))
    )
    # Flag geral de qualidade
    .withColumn(
        "flag_qualidade",
        F.when(
            F.col("flag_pk_valida") &
            F.col("flag_fk_valida") &
            F.col("flag_nota_valida"),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)
print("✅ Regras de validação aplicadas (Notas nulas aceitas como válidas)")

## 4️⃣ Separação de Dados Válidos e Inválidos

In [0]:
df_invalidos = (
    df_validacao
    .filter(F.col("flag_qualidade") == "ERRO")
    .withColumn(
        "motivo_rejeicao",
        F.concat_ws(
            "; ",
            F.when(~F.col("flag_pk_valida"), F.lit("cd_pesquisa NULL")),
            F.when(~F.col("flag_fk_valida"), F.lit("cd_chamado NULL")),
            F.when(~F.col("flag_nota_valida"), F.lit("nu_nota fora do range 1-5"))
        )
    )
    .withColumn("dt_validacao", F.lit(processing_timestamp))
    .drop(
        "flag_pk_valida",
        "flag_fk_valida",
        "flag_nota_valida",
        "data_processamento"
    )
)

## 5️⃣ Gravação: MERGE de Registros Válidos

In [0]:
# ---------------------------
# MERGE registros válidos na tabela principal
# ---------------------------
if total_validos > 0:
    # Criar temp view
    df_validos.createOrReplaceTempView("temp_pesquisa_validos")
    
    print(f"🔄 Executando MERGE de {total_validos} registros válidos em {TABLE_TARGET_VALIDOS}...")
    
    # MERGE SQL - Chave: cd_pesquisa
    spark.sql(f"""
        MERGE INTO {TABLE_TARGET_VALIDOS} AS target
        USING temp_pesquisa_validos AS source
        ON target.cd_pesquisa = source.cd_pesquisa
        
        WHEN MATCHED THEN
            UPDATE SET
                target.cd_chamado = source.cd_chamado,
                target.nu_nota = source.nu_nota,
                target.dt_ingestion = source.dt_ingestion,
                target.dc_origem = source.dc_origem
        
        WHEN NOT MATCHED THEN
            INSERT (
                cd_pesquisa,
                cd_chamado,
                nu_nota,
                dt_ingestion,
                dc_origem
            )
            VALUES (
                source.cd_pesquisa,
                source.cd_chamado,
                source.nu_nota,
                source.dt_ingestion,
                source.dc_origem
            )
    """)
    
    print(f"✅ MERGE concluído com sucesso!")
    
    # Mostrar estatísticas do MERGE
    merge_stats = spark.sql(f"""
        SELECT 
            version,
            timestamp,
            operationMetrics.numTargetRowsInserted as inseridos,
            operationMetrics.numTargetRowsUpdated as atualizados,
            operationMetrics.numOutputRows as total_afetados
        FROM (DESCRIBE HISTORY {TABLE_TARGET_VALIDOS})
        WHERE operation = 'MERGE'
        ORDER BY version DESC
        LIMIT 1
    """)
    
    if merge_stats.count() > 0:
        stats = merge_stats.collect()[0]
        print(f"\n📊 Estatísticas do MERGE:")
        print(f"   - Registros inseridos: {stats['inseridos']}")
        print(f"   - Registros atualizados: {stats['atualizados']}")
        print(f"   - Total afetados: {stats['total_afetados']}")
else:
    print("⚠️  Nenhum registro válido para inserir na Silver")

## 6️⃣ Gravação: APPEND de Registros Inválidos

In [0]:
if total_invalidos > 0:
    print(f"📝 Enviando {total_invalidos} registro(s) inválido(s) para auditoria...")
    
    # APPEND (acumula histórico de rejeições)
    (df_invalidos.write
        .format("delta")
        .mode("append")
        .saveAsTable(TABLE_TARGET_INVALIDOS))
    
    print(f"✅ Registros inválidos gravados em {TABLE_TARGET_INVALIDOS}")
    
else:
    print("🎉 Nenhum registro inválido - 100% de qualidade!")